In [8]:
import tensorflow as tf
import numpy as np, cv2, io, os, re, string, time, datetime
import seaborn as sns, sklearn
import matplotlib.pyplot as plt
from keras.layers import (TextVectorization)

In [3]:
#@ Downloading datasets:
!wget https://www.manythings.org/anki/fra-eng.zip

--2025-01-07 15:37:49--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  9.35MB/s    in 0.8s    

2025-01-07 15:37:54 (9.35 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]



In [4]:
!unzip '/content/fra-eng.zip' -d '/content/dataset' # -d flag specifies directories

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


#### Data Preprocessing

In [5]:
text_dataset=tf.data.TextLineDataset('/content/dataset/fra.txt') #each line is treated as separate string

In [6]:
for i in text_dataset.take(3):
  print(i)

tf.Tensor(b'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)', shape=(), dtype=string)
tf.Tensor(b'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)', shape=(), dtype=string)
tf.Tensor(b'Go.\tEn route !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)', shape=(), dtype=string)


In [12]:
#@ Setting up the Parameters:
VOCAB_SIZE=20000 #unique tokens from dataset, setting value 20000 for efficiency
ENGLISH_SEQUENCE_LENGTH=32 #max length of i/p sequence[in tokens]
FRENCH_SEQUENCE_LENGTH=32 #max len of o/p sequence[in tokens]
EMBEDDINGS_DIM=512 #size of vectors to represent tokens(as per paper)
BATCH_SIZE=128 #for data size processed during training

In [13]:
#@ for english word:
english_vectorize_layer=TextVectorization(
                      standardize='lower_and_strip_punctuation',
                      max_tokens=VOCAB_SIZE,
                      output_mode='int', #mapping wrt to the integer index
                      output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)

#@ for french word:
french_vectorize_layer=TextVectorization(
                       standardize='lower_and_strip_punctuation',
                       max_tokens=VOCAB_SIZE,
                       output_mode='int',
                       output_sequence_length=FRENCH_SEQUENCE_LENGTH
)

In [14]:
def selector(input_text):
  split_text=tf.strings.split(input_text, '\t')
  return {
      'input_1':split_text[0:1],
      'input_2':'starttoken' + split_text[1:2]
      }, split_text[1:2]+' endtoken'

In [21]:
text='hello\tprijal'
selector(text)


({'input_1': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'hello'], dtype=object)>,
  'input_2': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'starttokenprijal'], dtype=object)>},
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'prijal endtoken'], dtype=object)>)